In [ ]:
#An ECB/CBC detection oracle
#importing classes
import random
from random import randint
import os
import string
# The below file is given by OPEN SSL
!pip install pycrypto
from Crypto.Cipher import AES
from math import ceil
from binascii import unhexlify, b2a_base64, a2b_base64, hexlify
import chardet

In [ ]:
#intialization
ECB_MODE=1
CBC_MODE=0
Block_size=16

In [ ]:
def random_byte_create(lenofbytes):
  randomnumber=[]
  for i in range(lenofbytes):
    randomnumber.append(random.randint(0,255))
  return bytes(randomnumber)


In [ ]:
def encrypt_oracle(message):
  #returns byte of encypted message
  appendbefore=random_byte_create(random.randint(5,10))
  appendafter=random_byte_create(random.randint(5,10))
  #Generating block of size 16
  plaintext=pkcs7(appendbefore+message+appendafter,Block_size)
  key=random_byte_create(Block_size)
  type_of_encyption=random.randint(0,1)
  if type_of_encyption==ECB_MODE:
    #Encypt ECB
    cipher_object=AES.new(key,AES.MODE_ECB)
    return cipher_object.encrypt(plaintext),ECB_MODE

  elif type_of_encyption==CBC_MODE:
    #Encypt CBC
    IV=random_byte_create(Block_size)
    cipher_object=AES.new(key,AES.MODE_CBC,IV)
    return cipher_object.encrypt(plaintext),CBC_MODE


In [ ]:
#Detect the Encryption mechanism of the block
def guess_aes_ecb(ciphertext):
  #The logic is simple, In ECB the block content will be same , becuase for same message the cipher text will be same and not randomized
  #This formula only fails when messages are all unique
  blocks_of_ciphertext=[]
  number_of_blocks=len(ciphertext)//Block_size
  for i in range(number_of_blocks):
    blocks_of_ciphertext.append(ciphertext[i:i+Block_size])
  unique_num_blocks=set(blocks_of_ciphertext)
  #Returns True for ECB otherwise false
  return len(unique_num_blocks)<len(ciphertext)


In [ ]:
def decrypt_oracle():
  #decided a random plain text
  plaintext="Shristy"*(Block_size)
  plaintext=bytes(plaintext,'utf-8')
  ciphertext,mode=encrypt_oracle(plaintext)
  if guess_aes_ecb(ciphertext):
    return mode == ECB_MODE
  else:
    return mode == CBC_MODE


In [ ]:
#Testing the algorithm
if __name__== "__main__":
  rightans=0
  testcase = 2000
  for i in range(testcase):
    if decrypt_oracle():
      rightans+=1
  #Percentage of correct ans
  print((rightans/testcase)*100)





50.1
